#### Данное решение не является решением изначальной задачи. 

Здесь мы хотим узнать затраты на 1 итерацию запланированных ремонтов. Изначальная же задача состоит в том, чтобы для всего временного интервала и всех вагонов отдать даты ремонтов, которые могут происходить раньше запланированных.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import datetime
repair_is_valid_for = datetime.timedelta(days=365)*2
repare_types=['std_kap', 'std_ppr', 'std_dep', 'std_vogi']

In [3]:
# из примера, но мы этот брать не будем
#car_num = 57463085  # возьмем только один вагон 

car_num = 57463507  # возьмем только один вагон 
repares = pd.read_csv('dataset/Вагоны и плановые ремонты.csv')

for repair in repare_types:
    repares[repair] = pd.to_datetime(repares[repair])
repares["prev_kap"] = repares["std_kap"] - repair_is_valid_for
repares["prev_ppr"] = repares["std_ppr"] - repair_is_valid_for
repares["prev_dep"] = repares["std_dep"] - repair_is_valid_for
repares["prev_vogi"] = repares["std_dep"] - repair_is_valid_for

repares.to_json("repairs_date.json")
repare = repares[repares['car_num'] == car_num]
repare
repare_melt=repare.melt()
repare_melt=repare_melt[repare_melt["variable"].isin(repare_types)]
repare_melt["value"]=pd.to_datetime(repare_melt["value"])

In [4]:
movements = pd.read_csv('dataset/Передвижения вагонов по датам.csv')
movements["date"] = pd.to_datetime(movements["date"])
movement = movements[movements['car_num'] == car_num]
movement.head()

,car_num,date,station_id
687,57463507,2015-01-01,2
688,57463507,2015-01-25,6
689,57463507,2015-02-18,9
690,57463507,2015-03-14,5
691,57463507,2015-04-07,8


In [5]:
repare_prices = pd.read_csv('dataset/Плановые ремонты со стоимостями.csv')
repare_prices

,id,repair_name,repair_cost
0,1,Капитальный ремонт,94000000
1,2,Деповской ремонт,19000000
2,3,Плановый предупредительный ремонт,13000000
3,4,Вакуумная очистка и гидроиспытания (ВОиГИ),18000000


In [6]:
repare_prices = {'std_kap': 94000000, 
                 'std_ppr': 19000000, 
                 'std_dep': 13000000, 
                 'std_vogi': 18000000, 
                 'preparation': 3000000}
price_for_km = 1000

In [7]:
distances = pd.read_csv('dataset/Расстояния от станций до ремонтных депо.csv')
distances.head()

,station_id,name,depo_id,depo_name,distance_to_depo
0,1,Сосновка,1,Депо-Ленинское,10077
1,1,Сосновка,2,Депо-Комсомольское,10436
2,1,Сосновка,3,Депо-Молодежное,10522
3,2,Липовка,1,Депо-Ленинское,2084
4,2,Липовка,2,Депо-Комсомольское,3789


#### Посчитаем стоимость ремонтов, в случае, если делаем их по расписанию. 

Тут нужно сделать замечание, что мы считаем только 1 итерацию ремонтов. Правильнее будет посчитать все итерации: то есть если капитальный ремонт запланирован на 2021-08-08, но мы видим, что вагон эксплуатировался 2019 году, то 2019-08-08 у него должен был быть запланированный капитальный ремонт, т.к. ремонты утилизируются за 2 года

In [8]:
def get_station_from_date(date, car_num):
    for index, (car_num, station_date, station_id) in movements[movements['car_num'] == car].iterrows():
        if station_date < date:
            return station_id
    assert False, 'No stations before this date'

total_repare_price = 0
for repare_type in ['std_kap', 'std_ppr', 'std_dep', 'std_vogi']:
    repare_date = repares[repare_type].values[0]
    station_id = get_station_from_date(repare_date)
    min_distance_to_depo = distances[distances['station_id'] == 2]['distance_to_depo'].min()
    total_repare_price += 2 * min_distance_to_depo * price_for_km + \
            repare_prices[repare_type] + \
            repare_prices['preparation']

total_repare_price

TypeError: get_station_from_date() takes exactly 2 arguments (1 given)

## Возможные варианты ремонта

In [13]:
def get_station_from_date(date, car_num):
    for index, (car_num, station_date, station_id) in movements[movements['car_num'] == car_num].iterrows():
        if station_date < date:
            return station_id
    assert False, 'No stations before this date'

total_repare_price = 0
def repare_price(car_num, check_date, repare_types=repare_types):
    check_date = np.datetime64(check_date)
    total_repare_price = repare_prices['preparation']
    for repare_type in repare_types:
        station_id = get_station_from_date(check_date, car_num)
        min_distance_to_depo = distances[distances['station_id'] == 2]['distance_to_depo'].min()
        # Add path
        total_repare_price += 2 * min_distance_to_depo * price_for_km
        # Add repair cost
        if repares[repares["car_num"]==car_num][repare_type].values[0] <= check_date:
            total_repare_price += repare_prices[repare_type]
        else:
            total_repare_price += repare_prices[repare_type] * 5
    return total_repare_price

In [14]:
repare[repare_type].values[0]

numpy.datetime64('2021-07-19T00:00:00.000000000')

In [15]:
# Test
repare_price(car_num, repare[repare_type].values[0], repare_types=['std_kap', "std_ppr"]) > repare_price(car_num, repares[repare_type].values[0], repare_types=["std_ppr"])

True

In [16]:
repare_price(car_num, repare[repare_type].values[0], repare_types=['std_kap', "std_ppr"]) / 1000000 == 124

True

In [17]:
repares[repares["car_num"]==car_num]

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi,prev_kap,prev_ppr,prev_dep,prev_vogi
3,57463507,Газовые,15-78-5,2021-07-19,2019-03-18,2018-06-02,2019-05-18,2019-07-20,2017-03-18,2016-06-02,2016-06-02


In [20]:
from dateutil.parser import parse
min_date = pd.to_datetime(parse("2018-05-18"))
min_date = np.datetime64(min_date)
min_date

numpy.datetime64('2018-05-18T00:00:00.000000')

In [21]:
results = repares["std_ppr"] == min_date
repares["std_ppr"].min()

Timestamp('2015-04-05 00:00:00')

In [22]:
repare_price(car_num, min_date, repare_types=['std_ppr', 'std_dep', 'std_vogi']) / 1000000

265

In [24]:
repare_price(car_num, min_date+datetime.timedelta(days=0), repare_types=['std_ppr', 'std_dep', 'std_vogi']) / 1000000

TypeError: ufunc add cannot use operands with types dtype('<M8[us]') and dtype('O')

In [25]:
repares[repares["car_num"]==car_num][repare_type].values[0]

numpy.datetime64('2021-07-19T00:00:00.000000000')

In [26]:
repares[repares["car_num"]==car_num][repare_type].values[0] < min_date

False

In [27]:
repares[repares["car_num"]==car_num][repare_type].values[0]

numpy.datetime64('2021-07-19T00:00:00.000000000')

In [28]:
repare_price(car_num, min_date+pd.Timedelta('100 days'), repare_types=['std_ppr', 'std_dep', 'std_vogi']) / 1000000

213

In [29]:
import itertools

def powerset(L):
  pset = set()
  for n in xrange(len(L) + 1):
    for sset in itertools.combinations(L, n):
      #if sset:
      pset.add(sset)
  return pset

In [30]:
import itertools


In [31]:
from hyperopt import hp
# space = hp.choice('parameters', ['repare_types':powerset(repare_types), 'repare_types':powerset(repare_types)])

In [32]:
#hyperopt.fmin()

In [41]:
import time, pickle, hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x, car_num=car_num, check_date=min_date):
    return {
        'loss': repare_price(repare_types=x, car_num=car_num, check_date=repare_date),
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': time.time(),
        'other_stuff': {'type': None, 'value': [0, 1, 2]},
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }
trials = Trials()
space = hp.choice('repare_types',powerset(repare_types))


best = fmin(objective,
    space=space,
    algo=tpe.suggest,
    trials=trials,
    max_evals=100)

In [42]:
hyperopt.space_eval(space, best)

()

In [43]:
# for i in range(-10,100):
#     repare_date=min_date+pd.Timedelta(i, unit="d")
#     trials = Trials()
#     best = fmin(objective,
#         space=space,
#         algo=tpe.suggest,
#         trials=trials,
#         max_evals=20)
#     print(hyperopt.space_eval(space, best))

In [44]:
#space = hp.choice('parameters', parameters)

In [45]:
def inverse_mapping(f):
    return f.__class__(map(reversed, f.items()))

In [46]:
#word_counter_dictionary = {'first':1, 'second':2, 'third':3, 'fourth':2}
from collections import defaultdict

def inverse_mapping(dict_to_invert):
    # Mapping support repatable keys
    d = defaultdict(list)
    for key, value in dict_to_invert.iteritems():
        d[value].append(key)
    return(d)

In [52]:
def sum_multiple_repair(typles_dates_type, car_num = car_num):
    all_repairs_price = 0
    for date in typles_dates_type:
        repairs=typles_dates_type[date]
        all_repairs_price += repare_price(repare_types=repairs, car_num=car_num, check_date=date)
    return all_repairs_price

In [53]:
import time, pickle, hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def parameters(car_num):
    parameters = {}

    repare = repares[repares['car_num'] == car_num]
    repare_melt=repare.melt()
    repare_melt=repare_melt[repare_melt["variable"].isin(repare_types)]
    repare_melt["value"]=pd.to_datetime(repare_melt["value"])    

    for repare in repare_types:
        parameters[repare] = hp.choice(repare, pd.date_range(repare_melt["value"].min(), repare_melt["value"].max(), freq='D'))
    return parameters

big_space=parameters(car_num)
def objective(x, car_num=car_num, repare_date=min_date):
    print(car_num)
    price = sum_multiple_repair(inverse_mapping(x), car_num=car_num)
    return {
        'loss': price,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': time.time(),
        'other_stuff': {'type': None, 'value': [0, 1, 2]},
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }
trials = Trials()
best = fmin(objective,
    space=big_space,
    algo=tpe.suggest,
    trials=trials,
    max_evals=100)

57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507
57463507


In [59]:
params = inverse_mapping(hyperopt.pyll.stochastic.sample(parameters(car_num)))
params

defaultdict(list,
            {Timestamp('2019-06-07 00:00:00', freq='D'): ['std_vogi'],
             Timestamp('2020-06-13 00:00:00', freq='D'): ['std_kap'],
             Timestamp('2021-03-01 00:00:00', freq='D'): ['std_dep'],
             Timestamp('2021-07-04 00:00:00', freq='D'): ['std_ppr']})

In [60]:
sum_multiple_repair(params, car_num=car_num)

548672000

In [65]:
sum_multiple_repair(params, car_num=car_num) > sum_multiple_repair(inverse_mapping(hyperopt.space_eval(big_space, best)), car_num=car_num)

True

In [66]:
sum_multiple_repair(inverse_mapping(hyperopt.space_eval(big_space, best)), car_num=car_num)

545672000